<img src='../images/bildungscampus_logo.png' width="40%" align="left" />
<img src='../images/hhn.png' width="25%" align="right" />

# Schritt 1: Web Scraping and Storing
Masterarbeit - Sebastian Kahlert | Fakultät Wirtschaft und Verkehr | Wirtschaftsinformatik - Informationsmanagement und Data Science | WS 2021/22

<img src='../images/bar.png'/>

## Extraktion und Speicherung von Informationen aus Websiten

Bevor mit der eigentlichen Entwicklung der Chatbot-Kmponenten begonnen werden kann, müssen die Informationen, auf deren Basis der Chatbot arbeiten soll, beschafft und in einer Datenbank gespeichert werden. Das Wissen des Chatbots muss also beschafft werden. Im Rahmen dieser soll der Chatbot Prototyp für HHN Studierende ein Ansprechpartner sein, der Kontaktdaten und Forschungsgebiete der Professoren der HHN kennt. Darum werden diese Informationen im ersten Schritt durch Web Scraping extrahiert und in eine Datenbank gespeichert. 

### 1.1. Import der Bibliotheken 

Zum Web Scraping wird **BeautifulSoup** verwendet. BeautifulSoup ist eine freie Programmbibliothek für Screen Scraping, mit der sich XML- und HTML-Dokumente parsen lassen.

Zur Speicherung und späteren Abfrage der Informationen wird eine **SQLite-Datenbank** verwendet.  SQLite ist eine C-Bibliothek, die eine leichtgewichtige plattenbasierte Datenbank bereitstellt, die keinen separaten Serverprozess erfordert und den Zugriff auf die Datenbank mit einer nicht standardisierten Variante der SQL-Abfragesprache ermöglicht. Sie eignet sich be manchen Anwendungen zur internen Datenspeicherung. Es ist auch möglich, einen Prototyp einer Anwendung mit SQLite zu erstellen und den Code dann auf eine größere Datenbank wie PostgreSQL oder Oracle zu portieren.


In [32]:
# import of necessary libraries
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import sqlite3

### 1.2. Beschaffung von Namen der HHN Professoren 

Zunächst muss eine Liste mit möglichst vielen HHN Professoren erstellt werden. Diese Liste wird später durchiteriert, um für jeden Professor die HTML-Seite zu parsen und die benötigten Informationen extrahieren zu können. Da eine Lste der Professoren der HHN nicht fei zugänglich ist, wurden die Namen manuell in eine Textdatei geschrieben. Diese wird nun eingelsen und bearbeitet, sodass im Folgenden durchiteriert werden kann.

In [55]:
# read txt file and crate list with names
# remove /n and empty strings
text_file = open("test.txt", "r")
names = text_file.readlines()
for i in range(len(names)):
    names[i] = names[i].rstrip("\n")
names = list(filter(None, names))
text_file.close()

In [56]:
#names

In [124]:
url = "https://www.hs-heilbronn.de/carsten.lanquillon"

In [116]:
from selenium import webdriver
import time

# start web browser
driver=webdriver.Firefox(executable_path=r"geckodriver-v0.30.0-win64\geckodriver")

# get source code
driver.get(url)
html = driver.page_source
time.sleep(2)
doc = BeautifulSoup(html)
print(doc.prettify())

# close web browser
browser.close()

<html class="js-focus-visible" data-js-focus-visible="" data-react-helmet="lang" lang="de">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Scrivito by Infopark AG (scrivito.com)" name="generator"/>
  <title>
   Prof. Dr.-Ing. Carsten Lanquillon
  </title>
  <meta content="summary_large_image" data-react-helmet="true" name="twitter:card"/>
  <meta content="200" data-react-helmet="true" name="prerender-status-code"/>
  <meta content="article" data-react-helmet="true" property="og:type"/>
  <meta content="https://www.hs-heilbronn.de/carsten.lanquillon" data-react-helmet="true" property="og:url"/>
  <meta content="Fachgebiete
Business IntelligenceData ScienceMachine LearningKünstliche Intelligenz
Forschungsschwerpunkte
Data ScienceMachine..." data-react-helmet="true" name="twitter:description"/>
  <meta content="https://cdn0.scrvt.com/5b9bbd140a15e188780a6244

InvalidSessionIdException: Message: Tried to run command without establishing a connection


In [126]:
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

browser = webdriver.PhantomJS(r"phantomjs-2.1.1-windows\bin\phantomjs")
browser.get(url)
html = browser.page_source
doc = BeautifulSoup(html)
doc.prettify()

'<!DOCTYPE html>\n<html lang="de">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta content="width=device-width, initial-scale=1" name="viewport"/>\n  <meta content="Scrivito by Infopark AG (scrivito.com)" name="generator"/>\n  <title data-react-helmet="true">\n   Prof. Dr.-Ing. Carsten Lanquillon\n  </title>\n  <meta content="summary_large_image" data-react-helmet="true" name="twitter:card"/>\n  <meta content="200" data-react-helmet="true" name="prerender-status-code"/>\n  <meta content="article" data-react-helmet="true" property="og:type"/>\n  <meta content="https://www.hs-heilbronn.de/carsten.lanquillon" data-react-helmet="true" property="og:url"/>\n  <meta content="Fachgebiete\nBusiness IntelligenceData ScienceMachine LearningKünstliche Intelligenz\nForschungsschwerpunkte\nData ScienceMachine..." data-react-helmet="true" name="twitter:description"/>\n  <meta content="https://cdn0.scrvt.com/5b9bbd140a15e188780a6244ebe572d4/e0680f

### 1.3. Funktion für Kontaktdaten der Professoren 

Mithilfe von BeautifulSoup wird nun eine Funktion entwickelt, die durch die Liste der Professoren Namen iteriert, den HTML-Code der Profilseiten parst und die benötigten Informationen, wie Name und Email usw., extrahiert und in Listen speichert. Am Ende werden diese Listen in ein Dataframe umgewandelt, dass nun alle Kontaktdaten aller Professoren beinhaltet.


In [127]:
# implement function to extract contact data from the profile sites of the professors

def get_prof_info(list):
    title_list = []
    first_name_list = []
    last_name_list = []
    telephone_list = []
    email_list = []
    office_list = []
    postal_address_list = []
    prof_id_list = []
    
    for prof in list:      
        ### get prof_id ###
        prof_id = list.index(prof)+1
        prof_id_list.append(prof_id)

        url = "https://www.hs-heilbronn.de/"+prof
        
        # get source code
        browser = webdriver.PhantomJS(r"phantomjs-2.1.1-windows\bin\phantomjs")
        browser.get(url)
        html = browser.page_source
        doc = BeautifulSoup(html)
        doc.prettify()



        ### get title, first_name, last_name ### 
        full_name = doc.find(class_="mt-0 mb-3 p-0 sal-name").string

        # first_name
        first_name = full_name.split()[-2]
        first_name_list.append(first_name)

        # last_name
        last_name = full_name.split()[-1]
        last_name_list.append(last_name)

        # title
        title = full_name.replace(first_name, "")
        title = title.replace(last_name, "")
        title_list.append(title)

        ### get phone number ###
        telephone = doc.find(class_="mb-2").find(class_="color-teal").string
        telephone_list.append(telephone)

        ### get omly the first email ###
        email = doc.find_all(class_="mb-2")[1].find(class_="color-teal").string
        email_list.append(email)

        ### get office ###
        if "@" in doc.find_all(class_="mb-2")[2].span.string:
        # go to the next item to get the office
            office = doc.find_all(class_="mb-2")[3].span.string
        else:
            office = doc.find_all(class_="mb-2")[2].span.string
        office_list.append(office)

    

        
    df = pd.DataFrame({'prof_id': prof_id_list, 'title': title_list, 'first_name': first_name_list, 'last_name': last_name_list, 'telephone': telephone_list, 'email': email_list, 'office': office_list, 'postal_address': postal_address_list})
        # close web browser
    browser.close()
    return df

In [134]:
# implement function to extract contact data from the profile sites of the professors

def get_prof_info_test(list):
    title_list = []
    first_name_list = []
    last_name_list = []
    telephone_list = []
    email_list = []
    office_list = []
    postal_address_list = []
    prof_id_list = []
    
    for prof in list:      
        ### get prof_id ###
        prof_id = list.index(prof)+1
        prof_id_list.append(prof_id)

        url = "https://www.hs-heilbronn.de/"+prof
        
        # get source code
        browser = webdriver.PhantomJS(r"phantomjs-2.1.1-windows\bin\phantomjs")
        browser.get(url)
        time.sleep(5)
        html = browser.page_source
        doc = BeautifulSoup(html)
        print(doc.prettify())



        ### get title, first_name, last_name ### 
        full_name = doc.find(class_="mt-0 mb-3 p-0 sal-name").string

        # first_name
        first_name = full_name.split()[-2]
        first_name_list.append(first_name)

        # last_name
        last_name = full_name.split()[-1]
        last_name_list.append(last_name)

        # title
        title = full_name.replace(first_name, "")
        title = title.replace(last_name, "")
        title_list.append(title)

        ### get phone number ###
        telephone = doc.find(class_="mb-2").find(class_="color-teal").string
        telephone_list.append(telephone)

        ### get omly the first email ###
        email = doc.find_all(class_="mb-2")[1].find(class_="color-teal").string
        email_list.append(email)

        ### get office ###
        if "@" in doc.find_all(class_="mb-2")[2].span.string:
        # go to the next item to get the office
            office = doc.find_all(class_="mb-2")[3].span.string
        else:
            office = doc.find_all(class_="mb-2")[2].span.string
        office_list.append(office)

    

        
    df = pd.DataFrame({'prof_id': prof_id_list, 'title': title_list, 'first_name': first_name_list, 'last_name': last_name_list, 'telephone': telephone_list, 'email': email_list, 'office': office_list, 'postal_address': postal_address_list})
        # close web browser
    browser.close()
    return df

In [135]:
# calling the function
prof_info_df = get_prof_info_test(names)

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="Scrivito by Infopark AG (scrivito.com)" name="generator"/>
  <title>
   Your Scrivito powered site is loading ...
  </title>
  <link crossorigin="" href="https://api.scrivito.com" rel="preconnect"/>
  <link href="https://api.scrivito.com" rel="preconnect"/>
  <link href="https://cdn0.scrvt.com" rel="preconnect"/>
  <link href="/assets/index.css" rel="stylesheet"/>
  <script src="/js_snippets_head.js">
  </script>
  <script async="" src="https://client.crisp.chat/l.js">
  </script>
  <link crossorigin="" href="https://client.relay.crisp.chat" rel="dns-prefetch"/>
  <link crossorigin="" href="https://client.crisp.chat" rel="preconnect"/>
  <script async="" src="https://client.crisp.chat/static/javascripts/client.js?1960c46" type="text/javascript">
  </script>
  <link href="https:

AttributeError: 'NoneType' object has no attribute 'string'

In [ ]:
# show the df
prof_info_df

### 1.4. Funktion für Forschungsschwerpunkte

Nach dem gleichen Vorgehen, wie bei den Kontaktdaten, werden nun auch die Forschungschwerpunkte und danach die Studiengänge der Professoren extrahiert. 

In [11]:
# implement function to extract research areas from the profile sites of the professors
def get_research_area(list):
    research_area_list = []
    prof_id_list = []
    last_name_list = []
    

    ### get research areas - Fachgebiete ###
    
    for prof in list:
        ran = True
        prof_id = list.index(prof)+1
        
        url = "https://www.hs-heilbronn.de/"+prof
        result = requests.get(url).text
        doc = BeautifulSoup(result, "html.parser")
        parent = doc.find_all(class_="text-left")
        
                # last_name
        full_name = doc.find(class_="mt-0 mb-3 p-0 sal-name").string
        last_name = full_name.split()[-1]
        

        
        for i in range(len(parent)):
            if parent[i].string =="Fachgebiete":
                if parent[i+1].find_all("li") != None:
                    child = parent[i+1].find_all("li")
                    for l in range(len(child)):
                        research_area_list.append(child[l].string)
                        prof_id_list.append(prof_id)
                        last_name_list.append(last_name)
            elif parent[i].string =="Forschungsschwerpunkte" or parent[i].string =="Forschungsgebiete":
                if parent[i+1].find_all("li") != None:
                    child = parent[i+1].find_all("li")
                    for l in range(len(child)):
                        research_area_list.append(child[l].string)
                        prof_id_list.append(prof_id)
                        last_name_list.append(last_name)
            else:
                #if doc.select('ul.vertical-list') != None:
                if ran:
                    for i in range(len(doc.select('ul.vertical-list'))):
                        for l in range(len(doc.select('ul.vertical-list')[i])):
                            research_area_list.append(doc.select('ul.vertical-list')[i].find_all("li")[l].string)
                            prof_id_list.append(prof_id)
                            last_name_list.append(last_name)
                            ran = False


    df = pd.DataFrame({'prof_id': prof_id_list,"Name":last_name_list, 'research_area': research_area_list})           
    #print(len(prof_id_list))
    #print(len(research_area_list))
    
    return df

In [12]:
# calling the function
prof_research_area_df = get_research_area(more_prof_names)

In [13]:
# show the df
pd.set_option("display.max_rows", None, "display.max_columns", None)
prof_research_area_df

,prof_id,Name,research_area
0,1,Lanquillon,Business Intelligence
1,1,Lanquillon,Data Science
2,1,Lanquillon,Machine Learning
3,1,Lanquillon,Künstliche Intelligenz
4,1,Lanquillon,Big Data
5,1,Lanquillon,Business Intelligence
6,1,Lanquillon,Big Data Analytics
7,1,Lanquillon,Data Science
8,1,Lanquillon,Data Mining
9,1,Lanquillon,Text Mining


### 1.4. Funktion für Studiengänge

In [14]:
# implement function to extract studies from the profile sites of the professors
def get_study(list):
    study_list = []
    prof_id_list = []
    last_name_list = []
    

    ### get research areas - Fachgebiete ###
    
    for prof in list:
        prof_id = list.index(prof)+1
        
        url = "https://www.hs-heilbronn.de/"+prof
        result = requests.get(url).text
        doc = BeautifulSoup(result, "html.parser")
        parent = doc.find_all(class_="text-left")
        
                # last_name
        full_name = doc.find(class_="mt-0 mb-3 p-0 sal-name").string
        last_name = full_name.split()[-1]
        
        for i in range(len(doc.find_all(class_ = "color-teal"))):
            study = doc.find_all(class_ = "color-teal")[i].string
            if "+" not in study and  "@" not in study:
                study_list.append(study)
                prof_id_list.append(prof_id)
                last_name_list.append(last_name)
        
    df = pd.DataFrame({'prof_id': prof_id_list,"Name":last_name_list, 'study': study_list})           
    #print(len(prof_id_list))
    #print(len(research_area_list))
    
    return df

In [25]:
# calling the function
prof_study_df = get_study(more_prof_names)

In [27]:
# show the df
prof_study_df

,prof_id,Name,study
0,1,Lanquillon,Wirtschaftsinformatik
1,2,Stern,Wirtschaftsinformatik
2,2,Stern,Wirtschaftsinformatik - Informationsmanagement...
3,2,Stern,Wirtschaftsinformatik - Digitale Transformatio...
4,2,Stern,Fakultät Wirtschaft und Verkehr
5,3,Günther,Wirtschaftsinformatik
6,4,Beckmann,Wirtschaftsinformatik - Informationsmanagement...
7,4,Beckmann,Wirtschaftsinformatik
8,5,Schäffer,Wirtschaftsinformatik
9,6,Salmen,Wirtschaftsinformatik


### 1.5. Dataframes in sqlite Datenbank schreiben 

Diese drei DataFrames werden nun in die SQLite Datenbank gespeichert. Dabei wird jedes df in eine eigene Tabelle gespeichert um so die dritte Normalfunktion erfüllen zu können. Die Datenbank hat den Namen "PROF_INFO_DB".

**Tabellen der Datenbank PROF_INFO_DB:**<br>
PROF_INFO_TABLE --> Kontaktdaten der Professoren <br>
PROF_RESEARCH_AREA_TABLE --> Forschungsschwerpunkte der Professoren <br>
PROF_STUDY_TABLE --> Studiengänge der Professoren


In [16]:
# establish connection to DB, create the tables and write the dataframes into the tables
conn = sqlite3.connect("PROF_INFO_DB.db")
cur = conn.cursor()
prof_info_df.to_sql("PROF_INFO_TABLE", conn, index=False)
prof_research_area_df.to_sql("PROF_RESEARCH_AREA_TABLE", conn, index=False)
prof_study_df.to_sql("PROF_STUDY_TABLE", conn, index=False)
conn.close()

NameError: name 'prof_info_df' is not defined

#### SQL Queries testen

In [21]:
conn = sqlite3.connect("../PROF_INFO_DB.db")
cur = conn.cursor()
cur.execute("select research_area FROM PROF_RESEARCH_AREA_TABLE")
conn.commit()
rows = cur.fetchall()
conn.close()

#rows

### 1.6. Herausforderungen 

* Viele der extrahierten Professoren scheinen nicht mehr an der HHN tätig zu sein, da viele keine Profilseite auf der HHN Website haben. Demnach ist es notwendig auf anderen Websiten nach weiteren HHN Professoren zu suchen.
* Der Aufbau der Kontaktdaten ist auf den Profilseiten der Professoren fast immer gleich. Jedoch varriert die Darstellung der Forschungsschwerpunkte bei manchen Professoren stark vom "Standard". Das heißt die gesuchten Informationen sind bei manchen Professoren nicht in der gleichen HTML-Klasse zu finden wie bei den meisten. 
* Manche Professoren haben ihre Profilseite nicht gepflegt, weshalb hier manche Informationen fehlen.